# 5DEEP - Projet final : Classification des chants d'oiseaux

Pour rappel:

Il s'agit d'un problème de classification de chants d'oiseaux avec 5 classes, une pour chacune des espèces suivantes : 

- Bewick's Wren
- Northern Cardinal
- American Robin
- Song Sparrow
- Northern Mockingbird

Pour répondre à cette problématique, nous utiliserons des réseaux de neuronnes que nous préparerons et entraînerons grâce à Keras et Librosa dans ce notebook Jupyter.

## Etape 1: Analyse exploratoire et comprehension des données

Nous allons tout d'abord commencer par comprendre les différentes données mises à disposition, mais aussi comprendre l'interpretation de ces fichiers audio et leur traitement.

Quand nous récupérons des fichiers audio numérisés avec Librosa, nous avons :
- La fréquence: grave, aigue, etc (Hz)
- L'amplitude: Intensité/puissance du signal sonore (db)
- Nb de canaux: nb de flux différents (mono / stéreo , +)
- Fréquence d'échantillonage: la qualité audio (Hz ou kHz)

In [ ]:
import pandas as pd

bird_df = pd.read_csv("data/bird_songs_metadata.csv")

bird_df.head(10)

bird_df.info()
bird_df.isnull().sum()

print("Nombre d'observations:", len(bird_df))

# On retire 'subspecies' car trop corrélée avec 'species' et pas de sens pour notre classification
# 'recordist', 'license', 'remarks', 'source_url' n'apporte pas de sens pour notre classificiation

# On conserve le 'filename' pour retrouver l'enregistrement associé

# on retire 'species' car trop corrélée avec 'name' (même info mais scientifique vs commun)

# 'sound_type' -> On filtre ceux qui ne sont pas 'song'

# Notre variable cilble est 'species'

bird_df_song_only = bird_df[bird_df['sound_type'].str.contains("song", case=False, na=False)]

cols = ["id", "genus", "name", "country", "location", 
        "latitude", "longitude", "altitude", "time", "date", "filename", 'sound_type']
bird_df_useful = bird_df_song_only[cols]

# on vire les observations qui ne n'ont pas de fichier audio
bird_df_useful = bird_df_useful.dropna(subset=['filename'])

bird_df_useful.head(10)

In [ ]:
## Distribution & corrélations (6eme point)

import matplotlib.pyplot as plt

bird_df_useful['name'].value_counts().plot(kind='bar', figsize=(8,4))
plt.title("Nombre d'échantillons par espèce")
plt.xlabel("Espèce")
plt.ylabel("Nombre d'enregistrements")
plt.show()

actual_species = bird_df_useful['name'].unique()
print("Espèces trouvées :", actual_species)

class_counts = bird_df_useful['name'].value_counts()
print("Distribution des classes espèce:")
print(class_counts)



In [ ]:
## Test librosa

import librosa
import librosa.display
import numpy as np

sample_file = bird_df_useful['filename'].iloc[0]
path = f"data/wavfiles/{sample_file}"
y, sr = librosa.load(path, sr=22050, mono=True)

# forme du signal
plt.figure(figsize=(10,3))
librosa.display.waveshow(y, sr=sr)
plt.title(f"Waveform - {sample_file}")
plt.show()

# mel spectrogram
def extract_mel_spectrogram(file_path, sr=22050, n_mels=128, n_fft=2048, hop_length=512):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length)
    log_mel = librosa.power_to_db(mel, ref=np.max)
    return log_mel.T # shape (time, n_mels)

# mfcc 
def extract_mfcc(file_path, sr=22050, n_mfcc=40, n_fft=2048, hop_length=512):
    y, sr = librosa.load(file_path, sr=sr, mono=True)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc, n_fft=n_fft, hop_length=hop_length)
    return mfcc.T  # shape (time, n_mfcc)


# test de la fonction mel spectrogram sur le sample
mel_spec = extract_mel_spectrogram(f"data/wavfiles/{sample_file}")
plt.figure(figsize=(10,4))
librosa.display.specshow(mel_spec.T, sr=sr, hop_length=512,
                            x_axis="time", y_axis="mel")
plt.colorbar(format="%+2.0f dB")
plt.title("Mel Specrtrogram")
plt.show()

# test de la fonction mfcc sur le sample
mfcc_feat = extract_mfcc(f"data/wavfiles/{sample_file}")
plt.figure(figsize=(10,4))
librosa.display.specshow(mfcc_feat.T, sr=sr, hop_length=512,
                            x_axis="time")
plt.colorbar()
plt.title("MFCC")
plt.show()


def analyze_audio_properties(df, sample_size=50):
    """Analyse les propriétés audio sur un échantillon"""
    print(f"\n=== ANALYSE DES PROPRIÉTÉS AUDIO (échantillon de {sample_size}) ===")
    sample_df = df.sample(n=min(sample_size, len(df)), random_state=42)
    
    properties = []
    for idx, row in sample_df.iterrows():
        try:
            y, sr = librosa.load(f"data/wavfiles/{row['filename']}", sr=None)
            import soundfile as sf
            try:
                info = sf.info(f"data/wavfiles/{row['filename']}")
                channels = info.channels
            except:
                channels = 1 if len(y.shape) == 1 else y.shape[0]
            
            properties.append({
                'filename': row['filename'],
                'species': row['name'],
                'sample_rate': sr,
                'channels': channels,
                'duration': librosa.get_duration(y=y, sr=sr),
                'samples': len(y)
            })
        except Exception as e:
            print(f"Erreur avec {row['filename']}: {e}")
    
    return pd.DataFrame(properties)
audio_props = analyze_audio_properties(bird_df_useful, sample_size=100)
print(audio_props.describe())


## Encoding & Sépération des différents datasets

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical


labels = bird_df_useful['name'].values
encoder = LabelEncoder()
y_int = encoder.fit_transform(labels)
y = to_categorical(y_int)
print("content de y :", y[10:])
print("Shape de y :", y.shape)

# on recup pour le moment juste les filenames, on constuira notre set de features plus tard
# grace à librosa
X = bird_df_useful['filename'].values

# 70% train, 15% validation, 15% test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

print("Train :", len(X_train), "Validation :", len(X_val), "Test :", len(X_test))


## Encoding et pré-traitement

In [ ]:
## Pré-traitement des données audio

bird_df_useful_w_duration = bird_df_useful.copy()

# reucp la durée des enregistrements
durations = []
for f in bird_df_useful['filename']:
    y, sr = librosa.load(f"data/wavfiles/{f}", sr=None)
    durations.append(librosa.get_duration(y=y, sr=sr))
bird_df_useful_w_duration['duration'] = durations


def prepare_dataset(file_list, base_path="data/wavfiles/"):
    features = []
    for f in file_list:
        path = base_path + f
        mel_spec = extract_mel_spectrogram(path)
        features.append(mel_spec)
    return features

# Exemple pour le train
X_train_features = prepare_dataset(X_train)


print("Exemple de feature shape:", X_train_features[0].shape)
